In [1]:

import gzip
from collections import defaultdict
import math
import scipy.optimize
import numpy
import string
import random
import string
from sklearn import linear_model

In [2]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [3]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r


In [4]:
allRatings = []
for l in readCSV("train_Interactions.csv.gz"):
    allRatings.append(l)

In [5]:
train = allRatings[:190000]
valid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
usersPerBook = defaultdict(set)
booksPerUser = defaultdict(set)
for u,b,r in train:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))
    usersPerBook[b].add(u)
    booksPerUser[u].add(b)
    

In [6]:
# useful sets
ubdict = defaultdict(list)
userSet = set()
bookSet = set()
readSet = set()

for u,b,r in allRatings:
    userSet.add(u)
    bookSet.add(b)
    readSet.add((u,b))
    ubdict[u].append(b)

lUserSet = list(userSet)
lBookSet = list(bookSet)

In [7]:
def get(val):
    m=[]
    m.append(val[0])
    m.append(val[1])
    m.append(True)
    #m.append(val[2])
    return m

In [8]:
posvalid =[get(v) for v in valid]
postrain =[get(v) for v in train]

In [9]:
posvalid[0]

['u35176258', 'b30592470', True]

## Q1 Adding Negative labels to Validation set

In [10]:
def negbook(val):
    neg = []
    while len(neg)==0:
        b= random.choice(lBookSet)
        if b not in ubdict[val[0]]:
            neg.append(val[0])
            neg.append(b)
            neg.append(False)         
    return neg

In [11]:
negvalid = [negbook(v) for v in valid]

In [12]:
total_valid = posvalid + negvalid
#random.shuffle(total_valid)

In [13]:
y = [v[2] for v in total_valid]

## Combining Jaccard,cosine,Popularity

In [14]:
### Using logistic regression to combine the jaccard and most popularity,cosine 
negtrain = [negbook(v) for v in train]

In [15]:
total_train = postrain + negtrain
#random.shuffle(total_train)

In [16]:
len(total_train)

380000

In [66]:
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in train:
  bookCount[book] += 1
  totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalRead*0.58: break
    
    
   

In [67]:
pop_pre = []
for u,b,_ in total_train:
    if b in return1:
        pop_pre.append(True)
    else:
        pop_pre.append(False)

In [17]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom > 0:
        return numer/denom
    return 1

In [18]:
def mostSimilarFast(u,b):
    similarities = []   
    #b1- all users who have read b
    users = usersPerBook[b]
   

    for i2 in booksPerUser[u]:
        if i2 == b: continue
        sim = Jaccard(users, usersPerBook[i2])
        similarities.append((sim))
    similarities.sort(reverse=True)
    for x in similarities[:1]:
        if x > 0:
            return x
    return 0
                            

In [19]:
Jac_pre = []
for u,b,_ in total_train:
    temp = mostSimilarFast(u,b)
    Jac_pre.append(temp)
    
  

In [20]:
import math as mt
def cosine(s1, s2):
    numer = len(s1.intersection(s2))
    l1=len(s1)
    l2=len(s2)
    denom = mt.sqrt(l1) * mt.sqrt(l2)
    if denom > 0:
        return numer/denom
    return 1

In [21]:
def mostSimilarcosine(u,b):
    similarities_ = []   
    #b1- all users who have read b
    users_ = usersPerBook[b]
    
    #b1- all books read by user u
   
    for i2 in booksPerUser[u]:
        if i2 == b: continue
        sim_ = cosine(users_, usersPerBook[i2])
        similarities_.append((sim_))
    similarities_.sort(reverse=True)
    
    for x in similarities_[:1]:
        if x > 0:
            return x
    return 0
                            

In [22]:
cos_pre = []
for u,b,_ in total_train:
    temp = mostSimilarcosine(u,b)
    cos_pre.append(temp)
    
  

In [23]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom > 0:
        return numer/denom
    return 1

In [24]:
def mostSimilarFast_items(u,b):
    simil = []   
    #b1- all books read by user u 
    books = set(ratingsPerUser[u])
    
    for i2 in ratingsPerItem[b]:
        if i2 == u: continue
        sim = Jaccard(books, set(ratingsPerUser[i2]))
        simil.append((sim))
    simil.sort(reverse=True)
    for x in simil[:1]:
        if x > 0 :
            return x
    return 0
                       

In [25]:
Jac2_pre = []
for u,b,_ in total_train:
    temp = mostSimilarFast_items(u,b)
    Jac2_pre.append(temp)
    

In [26]:
## adding length of ratings

lenR = []
for u,b,_ in total_train:
    te=0
    temp = len(ratingsPerItem[b])
    if temp > te :
        lenR.append(temp)
    else:
        lenR.append(0)

In [27]:
import pandas as pd

In [68]:
df4 = pd.DataFrame(zip(pop_pre,Jac_pre,cos_pre,Jac2_pre,lenR))

In [28]:
df5 = pd.DataFrame(zip(Jac_pre,cos_pre,Jac2_pre,lenR))

In [146]:
df8= pd.DataFrame(zip(Jac2_pre,lenR))

In [147]:
x_train = df8

In [148]:
y = [d[2] for d in total_train]

In [217]:
from sklearn import linear_model
model = linear_model.LogisticRegression(C=0.00003)

In [218]:
model.fit(x_train,y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=3e-05, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [77]:
def total(v):
    total_comp=[]
    #print(v)
    h = mostSimilarFast(v[0],v[1])
    c = mostSimilarcosine(v[0],v[1])
    i = mostSimilarFast_items(v[0],v[1])

    
    if v[1] in return1:
        r = 1
    else:
        r = 0
        
    temp = len(ratingsPerItem[v[1]])
    if temp > 0 :
        j = temp
    else:
        j = 0
        
    #s=0
    #for u,r in ratingsPerItem[v[1]]:
       #s+= r
    #if s > 0 and temp > 0 :
        #a = s/temp
    #else:
        #a =0
        
    #total_comp.append(v[0])
    #total_comp.append(v[1])
    total_comp.append(r)
    total_comp.append(h)
    total_comp.append(c)
    total_comp.append(i)
    total_comp.append(j)
    #total_comp.append(a)
    return total_comp

In [78]:
v_reg =[total(k) for k in total_valid]

In [151]:
def feature(d):
    feat = [d[3],d[4]]
    return feat

In [152]:
x_valid = [feature(d) for d in v_reg]

In [153]:
y_valid = [d[2] for d in total_valid]

In [219]:
pred = model.predict(x_valid)

In [220]:
correct = pred == y_valid

In [221]:
# accuracy
sum(correct) / len(correct)

0.6555

## Making predictions on test data

In [122]:
#test_predict
test_pop_pre = []
test_jac_pre =[]
test_cos_pre= []
test_jac2_pre= []

test_len =[]

predictions = open("predictions_Read.txt", 'w')
for l in open("pairs_Read.txt"):
    if l.startswith("userID"):continue
    
    u,b = l.strip().split('-')
    
    if b in return1:
        test_pop_pre.append(1)
    else:
        test_pop_pre.append(0)
          
    t1 = mostSimilarFast(u,b)
    test_jac_pre.append(t1) 
        
    
    t2 = mostSimilarcosine(u,b)
    test_cos_pre.append(t2)
    
    t3 = mostSimilarFast_items(u,b)
    test_jac2_pre.append(t3)
    
    
    temp = len(usersPerBook[b])
    if temp > 0 :
        test_len.append(temp)
    else:
        test_len.append(0)
    
    
        
df_test = pd.DataFrame(zip(test_pop_pre,test_jac_pre,test_cos_pre,test_jac2_pre,test_len))

In [123]:
y_test= model.predict(df_test)

In [124]:
prob1= model.predict_proba(df_test)[:,1]

In [127]:
ub5dict = defaultdict(list)

userSet1 = set()

k=0
for l in open("pairs_Read.txt"):
    if l.startswith("userID"):
        continue
    u,b = l.strip().split('-')
    
    p=prob1[k]
    ub5dict[u].append((p,b))
    userSet1.add(u)
    k+=1

userlist1 = list(userSet1)

In [128]:
rank = defaultdict(list)
for u in userlist1:
    ub5dict[u].sort()
    ub5dict[u].reverse()
    l=len(ub5dict[u])
    i=0
    for p,b in ub5dict[u]:
        if i < l/2 :
            rank[u].append((b,1))
        else:
            rank[u].append((b,0))
        i +=1
        

In [129]:

predictions = open("predictions_Read.txt", 'w')
for l in open("pairs_Read.txt"):
    if l.startswith("userID"):
    #header
        predictions.write(l)
        continue
    u,b = l.strip().split('-')
    
    for b1,p in rank[u]:
        if b1 == b:
            predictions.write(u + '-' + b + ","+str(p)+"\n")

    
predictions.close()